In [1]:
from os import listdir
from os.path import isfile, join

import csv
import pandas as pd

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from spotipy.oauth2 import SpotifyOAuth

import requests
import re

In [2]:
#BOOKS 2 NOOKS ACCOUNT:
#user: Books2nooks@outlook.com
#pass: strugglebus6242gang

In [3]:
bookName = 'bookTest'

inputDict = {}

inputDict['bookTest - default'] = [
                                    "spotify:track:5WZDIh4WVkcHItK7y9E8Fl",
                                    "spotify:track:2rf9i0Enr8cw1JRME8Rsvq"
                                    ]

inputDict['bookTest - rock'] = [
                                "spotify:track:4ShMTKXex9FTogbRxbDxNF",
                                "spotify:track:2rhIfaoRGTAjC89HBhmb3R"
                                ]

In [4]:
#Set up all the keys and stuff
SPOTIFY_CLIENT_ID =""
SPOTIFY_SECRET =""
REDIRECT_URL = "http://localhost:3000/"

#set up spotipy manager
sp = spotipy.Spotify(
    auth_manager=SpotifyOAuth(
        scope="playlist-modify-public",
        redirect_uri=REDIRECT_URL,
        client_id=SPOTIFY_CLIENT_ID,
        client_secret=SPOTIFY_SECRET,
        cache_path="token.txt"
    )
)

#This is books2nooks's id
user_id = "31bezwu6mascvq773ob6sb7bikjy"

In [5]:
def getLinks(bookName, inputDict):
    #cut out uri header
    for key in inputDict:
        spotify_song_uris = []

        for uri in inputDict[key]:
            spotifyHeader = 'spotify:track:'

            cutUri = uri[len(spotifyHeader):]

            spotify_song_uris.append(cutUri)

        inputDict[key] = spotify_song_uris

    #loop through existing playlists to make sure that we dont create duplicate playlists
    os = 0
    stop = False

    inputKeys = list(inputDict.keys())
    holder = [0] * len(inputKeys)

    links = dict(zip(inputKeys, holder))

    while True:
        segment = sp.user_playlists(user_id, limit = 50, offset = os)

        numItems = len(segment['items'])

        for item in segment['items']:
            if item['name'] in inputDict:
                inputDict.pop(item['name'])

                links[item['name']] = item["external_urls"]['spotify']

                if len(inputDict) == 0:
                    stop = True
                    break

        if stop:
            break

        if numItems < 50 or segment['items'] == []:
            break

        os += 50

    if not stop:
        desc = "Generated playlist for " + bookName

        for key in inputDict:
            playlist = sp.user_playlist_create(user=f"{user_id}", name = key, public=True,
                                                  description=desc)

            playlistId = playlist["id"]

            link = playlist['external_urls']['spotify']

            sp.user_playlist_add_tracks(user=f"{user_id}", playlist_id=playlistId, tracks=inputDict[key])

            links[key] = link

        return links
    else:
        print('playlist exists already, here is link:')
        return links

In [6]:
getLinks(bookName, inputDict)

{'bookTest - default': 'https://open.spotify.com/playlist/6FwmzzIqG6M5nxKLG0KHDi',
 'bookTest - rock': 'https://open.spotify.com/playlist/5rDhjMetuODibS34iQaCDC'}